In [252]:
import os
import math
import numpy as np
import pandas as pd
import datetime
import time
import random
from datetime import date
import pandas_ta as ta
from ta.volatility import BollingerBands
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from yahoo_fin import stock_info as si
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
US_BUSINESS_DAY = CustomBusinessDay(calendar=USFederalHolidayCalendar())

In [253]:
def load_historic_data(symbol):
    today = datetime.date.today()
    today_str = today.strftime("%Y-%m-%d")
    #  Get last year's data
    start_date = today - (251 * US_BUSINESS_DAY)
    start_date_str = datetime.datetime.strftime(start_date, "%Y-%m-%d")
    # Download data from Yahoo Finance
    try:
        df = si.get_data(symbol, start_date=start_date_str, end_date=today_str, index_as_date=False)
        return df
    except:
        print('Error loading stock data for ' + symbol)
        return None
    
df = load_historic_data('TSLA')


#  Get the last year of data
df = df.tail(251)
df.reset_index(inplace=True)

In [254]:
def calculate_bollinger_bands(df):
    # Initialize Bollinger Bands Indicator
    indicator_bb = BollingerBands(close=df["close"], window=20, window_dev=2)

    # Add Bollinger Bands features
    df['BB_mid'] = indicator_bb.bollinger_mavg()
    df['BB_high'] = indicator_bb.bollinger_hband()
    df['BB_low'] = indicator_bb.bollinger_lband()

    return df

In [255]:
def apply_strategy_rules(df):
    #  Entry Rule 1: Close price below Low Bollinger Band
    df['BB_entry_signal'] = np.where((df["close"] < df["BB_low"]) & (df["close"].shift() >= df["BB_low"]), 1, 0)
    
    #  Exit rule
    df['BB_exit_signal'] = np.where((df["close"] > df["BB_high"]) & (df["close"].shift() <= df["BB_high"]), 1, 0)

    return df

In [256]:
def execute_strategy(df):
    close_prices = df['close']
    BB_entry_signals = df['BB_entry_signal']
    BB_exit_signals = df['BB_exit_signal']
    entry_prices = []
    exit_prices = []
    entry_signal = 0
    exit_signal = 0
    buy_price = -1
    hold = 0
    
    for i in range(len(close_prices)):
        #  Check entry and exit signals
        if BB_entry_signals[i] == 1:
            entry_signal = 1
        else:
            entry_signal = 0
        if BB_exit_signals[i] == 1:
            exit_signal = 1
        else:
            exit_signal = 0
            
        #  Add entry prices
        if hold == 0 and entry_signal == 1:
            buy_price = close_prices[i]
            entry_prices.append(close_prices[i])
            exit_prices.append(np.nan)  
            entry_signal = 0
            hold = 1
        #  Evaluate exit strategy
        elif (hold == 1 and exit_signal == 1 or (hold == 1 and close_prices[i] <= buy_price * 0.95)):
            entry_prices.append(np.nan)
            exit_prices.append(close_prices[i]) 
            exit_signal = 0
            buy_price = -1
            hold = 0
        else:
            #  Neither entry nor exit
            entry_prices.append(np.nan) 
            exit_prices.append(np.nan) 
            
    return entry_prices, exit_prices

In [257]:
def plot_graph(df, entry_prices, exit_prices):
    bb_high = df['BB_high']
    bb_mid = df['BB_mid']
    bb_low = df['BB_low']
    fig = make_subplots(rows=1, cols=1)

    #  Plot close price
    fig.add_trace(go.Line(x = df.index, y = df['close'], line=dict(color="blue", width=1), name="Close"), row = 1, col = 1)
    
    #  Plot bollinger bands
    fig.add_trace(go.Line(x = df.index, y = bb_high, line=dict(color="#ffdf80", width=1), name="BB High"), row = 1, col = 1)
    fig.add_trace(go.Line(x = df.index, y = bb_mid, line=dict(color="#ffd866", width=1), name="BB Mid"), row = 1, col = 1)
    fig.add_trace(go.Line(x = df.index, y = bb_low, line=dict(color="#ffd24d", width=1), name="BB Low"), row = 1, col = 1)
    
    #  Add buy and sell indicators
    fig.add_trace(go.Scatter(x=df.index, y=entry_prices, marker_symbol="arrow-up", marker=dict(
        color='green',
    ),mode='markers',name='Buy'))
    fig.add_trace(go.Scatter(x=df.index, y=exit_prices, marker_symbol="arrow-down", marker=dict(
        color='red'
    ),mode='markers',name='Sell'))
    
    fig.update_layout(
        title={'text':'BB + Stop Loss', 'x':0.5},
        autosize=False,
        width=800,height=400)
    fig.update_yaxes(range=[0,1000000000],secondary_y=True)
    fig.update_yaxes(visible=False, secondary_y=True)  #hide range slider
    
    fig.show()
    

In [258]:
def calculate_buy_hold_profit(investment, df):
    print(df)
    close_prices = df['close']
    buy_quantity = investment / close_prices[0]
    print('start ',close_prices[0])
    print('end ',close_prices[len(close_prices)-1])
    sell_amount = buy_quantity * close_prices[len(close_prices)-1]
    profit = sell_amount - investment
    return profit
   

In [259]:
def calculate_strategy_profit(investment, entry_prices, exit_prices):
    entry_price = 0
    hold = 0
    total_profit = 0
    quantity = 0
    available_funds = investment
    purchase_amount = 0
    
    for i in range(len(entry_prices)):
        current_entry_price = entry_prices[i]
        current_exit_price = exit_prices[i]
        
        if not math.isnan(current_entry_price) and hold == 0:
            entry_price = current_entry_price
            quantity = available_funds / entry_price
            purchase_amount = quantity * entry_price
            hold = 1
        elif hold == 1 and not math.isnan(current_exit_price):
            hold = 0
            sales_amount = quantity * current_exit_price
            profit_or_loss = sales_amount - purchase_amount
            available_funds = available_funds + profit_or_loss
            total_profit += profit_or_loss
        
    return total_profit        

In [260]:
#  Perform analysis
investment = 1000
df = load_historic_data('BKNG')
df.reset_index(inplace=True)
df = calculate_bollinger_bands(df)
df = apply_strategy_rules(df)
entry_prices, exit_prices = execute_strategy(df)
profit_or_loss = calculate_strategy_profit(investment, entry_prices, exit_prices)
buy_hold_profit = calculate_buy_hold_profit(investment, df)
plot_graph(df, entry_prices, exit_prices)

profit_or_loss:  14.101532536596437
     index       date         open         high          low        close  \
0        0 2021-04-16  2443.729980  2489.409912  2435.830078  2476.899902   
1        1 2021-04-19  2480.429932  2488.320068  2443.110107  2456.199951   
2        2 2021-04-20  2433.750000  2436.489990  2317.290039  2345.000000   
3        3 2021-04-21  2359.919922  2365.989990  2343.060059  2357.729980   
4        4 2021-04-22  2353.719971  2395.000000  2343.000000  2347.050049   
..     ...        ...          ...          ...          ...          ...   
249    249 2022-04-11  2150.000000  2209.560059  2144.949951  2169.209961   
250    250 2022-04-12  2196.949951  2232.370117  2172.820068  2181.379883   
251    251 2022-04-13  2192.250000  2272.310059  2190.689941  2260.110107   
252    252 2022-04-14  2277.000000  2307.459961  2213.449951  2227.290039   
253    253 2022-04-18  2210.000000  2226.179932  2173.000000  2199.310059   

        adjclose  volume ticker       B

C:\Users\dutch\anaconda3\envs\tradesystem1\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [261]:
def perform_analysis(symbol, df, investment):
    df = df.reset_index()
    df = calculate_bollinger_bands(df)
    df = apply_strategy_rules(df)
    
    entry_prices, exit_prices = execute_strategy(df)
    profit_or_loss = calculate_strategy_profit(investment, entry_prices, exit_prices)
    buy_hold_profit = calculate_buy_hold_profit(investment, df)
    return profit_or_loss, buy_hold_profit

In [262]:
# Backtesting using NASDAQ 100
nasdaq_100_df = pd.read_csv('https://raw.githubusercontent.com/justmobiledev/python-algorithmic-trading/main/data/nasdaq_100.csv')
nasdaq_100 = nasdaq_100_df['Symbol'].to_numpy()

In [263]:
#  Backtesting
total_strategy_profit = 0
total_buy_hold_profit = 0
for symbol in nasdaq_100:
    df = load_historic_data(symbol)
    if df is None or df.empty:
        continue
    df.reset_index(inplace=True)
    
    #  Random interval between remote fetch to avoid spam issues
    random_secs = random.uniform(0, 1)
    time.sleep(random_secs)
    
    #  Run backtest
    profit, buy_hold_profit = perform_analysis(symbol, df, investment=investment) 
    print(f"Backtest profit for symbol {symbol}: ${math.trunc(profit)}, buy & hold: ${math.trunc(buy_hold_profit)}")
    total_strategy_profit += profit
    total_buy_hold_profit += buy_hold_profit
  
print(f"\nAvg strategy profit per stock: ${math.trunc(total_strategy_profit / len(nasdaq_100))}")
print(f"\nAvg buy & hold profit per stock: ${math.trunc(total_buy_hold_profit / len(nasdaq_100))}")

'\ntotal_strategy_profit = 0\ntotal_buy_hold_profit = 0\nfor symbol in nasdaq_100:\n    df = load_historic_data(symbol)\n    if df is None or df.empty:\n        continue\n    df.reset_index(inplace=True)\n    \n    #  Random interval between remote fetch to avoid spam issues\n    random_secs = random.uniform(0, 1)\n    time.sleep(random_secs)\n    \n    #  Run backtest\n    profit, buy_hold_profit = perform_analysis(symbol, df, investment=investment) \n    print(f"Backtest profit for symbol {symbol}: ${math.trunc(profit)}, buy & hold: ${math.trunc(buy_hold_profit)}")\n    total_strategy_profit += profit\n    total_buy_hold_profit += buy_hold_profit\n  \nprint(f"\nAvg strategy profit per stock: ${math.trunc(total_strategy_profit / len(nasdaq_100))}")\nprint(f"\nAvg buy & hold profit per stock: ${math.trunc(total_buy_hold_profit / len(nasdaq_100))}")\n'